In [51]:
import pandas as pd

from datetime import date

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)

In [100]:
df_Xcite = pd.read_excel('Product_Catalog_Outputs\Xcite_product_catalog.xlsx')
df_Zain = pd.read_excel('Product_Catalog_Outputs\Zain_product_catalog.xlsx')

df_Xcite['name_split'] = df_Xcite['product_name'].str.lower().str.split(" ").apply(lambda x: set(x))
df_Zain['name_split'] = df_Zain['product_name'].str.lower().str.split(" ").apply(lambda x: set(x))

df_Xcite['price_float'] = df_Xcite['product_price'].str.split(' ').str[0].str.replace(',', '').astype(float)
df_Zain['price_float'] = df_Zain['product_price_cash'].str.split(' ').str[1].str.replace(',', '').astype(float)

Unique_ID_fields_Zain = df_Zain.loc[:,['name_split', 'product_brand', 'price_float']]
Unique_ID_fields_Xcite = df_Xcite.loc[:,['name_split', 'product_brand', 'price_float', 'in_out_stock']]

In [68]:
matching = [
    # match list
    [' '.join(list(name_list_zain)), ' '.join(list(name_list_xcite)), Unique_ID_fields_Zain['price_float'][index_zain], Unique_ID_fields_Xcite['price_float'][index_xcite]]
    
    # name_split iteration
    for index_zain, name_list_zain in enumerate(Unique_ID_fields_Zain['name_split'])
    for index_xcite, name_list_xcite in enumerate(Unique_ID_fields_Xcite['name_split'])
    
    # condition filters (at least 70% intersection, matching brand name, within 50% +/- price agreement, 'in stock')
    if name_list_zain.intersection(name_list_xcite) and len(name_list_zain.intersection(name_list_xcite))/len(name_list_zain) >= 0.7
    and str(Unique_ID_fields_Zain['product_brand'][index_zain]).lower() == str(Unique_ID_fields_Xcite['product_brand'][index_xcite]).lower()
    and Unique_ID_fields_Zain['price_float'][index_zain] >= Unique_ID_fields_Xcite['price_float'][index_xcite] * 0.5
    and Unique_ID_fields_Zain['price_float'][index_zain] <= Unique_ID_fields_Xcite['price_float'][index_xcite] * 1.5
    and Unique_ID_fields_Xcite['in_out_stock'][index_xcite] == 'in_stock'
    # ADD ZAIN IN_OUT_STOCK
    # ENRICH ZAIN NAMING CONVENTIONS AND DATA SET
]

In [90]:
percent_intersection_list = [
    round((len(name_list_zain.intersection(name_list_xcite)) / len(name_list_zain))*100,2)
    for index_zain, name_list_zain in enumerate(Unique_ID_fields_Zain['name_split'])
    for index_xcite, name_list_xcite in enumerate(Unique_ID_fields_Xcite['name_split'])
    if (
        len(name_list_zain.intersection(name_list_xcite)) / len(name_list_zain) >= 0.7
        and str(Unique_ID_fields_Zain['product_brand'][index_zain]).lower() == str(Unique_ID_fields_Xcite['product_brand'][index_xcite]).lower()
        and Unique_ID_fields_Zain['price_float'][index_zain] >= Unique_ID_fields_Xcite['price_float'][index_xcite] * 0.5
        and Unique_ID_fields_Zain['price_float'][index_zain] <= Unique_ID_fields_Xcite['price_float'][index_xcite] * 1.5
        and Unique_ID_fields_Xcite['in_out_stock'][index_xcite] == 'in_stock'
    )
]


In [95]:
match_df = pd.DataFrame(matching)

In [96]:
match_df.columns = ['zain_product', 'xcite_product', 'zain_price', 'xcite_price']

In [97]:
 match_df['price_difference [%]'] = round(((match_df['zain_price'] - match_df['xcite_price'])/match_df['zain_price'])*100,2)

In [98]:
match_df['percent_intersection [%]'] = percent_intersection_list

In [99]:
 match_df.sort_values('price_difference [%]')

,zain_product,xcite_product,zain_price,xcite_price,price_difference [%],percent_intersection [%]
72,series watch 8,steel loop 8 case apple + 41mm gold cellular with watch milanese gps series stainless,121.00,239.90,-98.26,100.00
73,series watch 8,steel graphite loop 8 case apple + 41mm cellular with watch milanese gps series stainless,121.00,239.90,-98.26,100.00
71,series watch 8,steel sport band white 8 case apple - + 41mm cellular with watch silver gps series regular stainless,121.00,220.90,-82.56,100.00
42,"10th 10.9"" gen ipad",10.9-inch apple - cellular silver gen ipad 10th 256gb,137.00,249.90,-82.41,75.00
43,"10th 10.9"" gen ipad",10.9-inch apple - blue cellular gen ipad 10th 256gb,137.00,249.90,-82.41,75.00
31,s23 galaxy,phone ultra - s23 galaxy samsung 512gb green,230.00,389.90,-69.52,100.00
36,s23 galaxy,phone ultra - s23 galaxy cream samsung 512gb,230.00,389.90,-69.52,100.00
35,s23 galaxy,phone black ultra - s23 galaxy phantom samsung 512gb,230.00,389.90,-69.52,100.00
17,matebook d15,"core 15.6-inch, 16gb bohre-wfe9al matebook 512gb ram, huawei - laptop, d15 windows silver i7, 11 intel ssd, 11th gen home,",160.00,254.90,-59.31,100.00
51,"10th 10.9"" gen ipad",10.9-inch pink apple - 64gb cellular gen ipad 10th,137.00,199.90,-45.91,75.00


In [53]:
match_df.to_excel(f'Analytics\matches_{date.today().strftime("%Y_%m_%d")}.xlsx')